# Using Jurassic-2 Grande Instruct on SageMaker through Model Packages

This sample notebook shows you how to deploy **Jurassic-2 Grande Instruct** using Amazon SageMaker.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
1. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   1. [Interact with the model](#B.-Interact-with-the-model)
   1. [Give it an instruction](#C.-Give-it-an-instruction)
   1. [Create your own product description!](#D.-Create-your-own-product-description!)
1. [Clean-up](#3.-Clean-up)
   1. [Delete the endpoint](#A.-Delete-the-endpoint)
   1. [Delete the model](#B.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select model package
Confirm that you received this notebook from the model catalog in SageMaker AWS Management Console.

In [5]:
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "us-west-1": "arn:aws:sagemaker:us-west-1:382657785993:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "eu-west-3": "arn:aws:sagemaker:eu-west-3:843114510376:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "eu-north-1": "arn:aws:sagemaker:eu-north-1:136758871317:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ap-southeast-1": "arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ap-southeast-2": "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "sa-east-1": "arn:aws:sagemaker:sa-east-1:270155090741:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378"
}

In [6]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
import sagemaker as sage
import boto3

### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [7]:
boto3.__version__

'1.26.111'

### Install ai21 python SDK

In [8]:
! pip install -U "ai21[SM]"
import ai21


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [9]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [10]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [15]:
endpoint_name = "j2-grande-instruct"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g5.12xlarge"
)

### A. Create an endpoint

In [17]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=endpoint_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

---------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Interact with the model

#### Be specific in your prompt
You can see that the model generated several versions. There may be times when you are looking for a stricter format - such as a single paragraph. Just ask the model to stick to it:

In [22]:
instruction = """
Write an engaging description for the Xiaomi Annual Report. Make sure to include the following key numbers in the description.
Revenue result in number, Profit resultsin number

2021 2020
(RMB in millions)
Revenue 328,309.1 245,865.6
Cost of sales (270,048.2) (209,113.8)
Gross profit 58,260.9 36,751.8
Research and development expenses (13,167.1) (9,256.1)
Selling and marketing expenses (20,980.8) (14,539.4)
Administrative expenses (4,738.9) (3,746.4)
Fair value changes on investments measured at fair value
through profit or loss 8,132.1 13,173.5
Share of net profits of investments accounted for
using the equity method 275.0 1,380.9
Other income 826.9 642.9
Other losses, net (2,579.5) (372.5)
Operating profit 26,028.6 24,034.7
Finance costs, net (1,611.6) (2,401.3)
Profit before income tax 24,417.0 21,633.4
Income tax expenses (5,133.8) (1,320.7)
Profit for the year 19,283.2 20,312.7
Non-IFRS Measure: Adjusted net profit 22,039.5 13,006.4

Description:

"""

response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt=instruction,
                                   maxTokens=400,
                                   temperature=0.5,
                                   numResults=1)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

Xiaomi's Annual Report for 2021 provides a detailed overview of the company's financial performance and operations for the year. The report includes key financial metrics such as revenue, gross profit, operating expenses, and net profit.

Xiaomi's revenue grew significantly in 2021, driven by strong sales of smartphones and other consumer electronics products. Gross profit also increased, driven by higher revenue and lower cost of sales.

Research and development expenses increased slightly in 2021, driven by investments in new technologies and product development. Selling and marketing expenses increased slightly, driven by increased investments in advertising and promotion.

Administrative expenses decreased slightly in 2021, driven by lower overhead costs. Fair value changes on investments measured at fair value through profit or loss decreased in 2021, driven by a decline in the value of Xiaomi's investment portfolio.

Share of net profits of investments accounted for using the equ

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [ ]:
model.delete_model()